In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

Cities available for PG/Coliving : Noida Delhi Gurgaon Bangalore Indore Pune Chennai Mumbai Hyderabad Ahmedabad

Cities available for BHK/Rooms : Noida Ghaziabad Gurgaon Delhi Bangalore Chennai Mumbai Indore Nagpur

BHK/ROoms : https://www.roomsoom.com/flat/property-listing?lat=&lang=&city=Noida&type=flat

PG/Coliving : https://www.roomsoom.com/pg/property-listing?lat=&lang=&city=Noida&type=pg

In [2]:
# pg/co-living
city_pg = ['Noida', 'Delhi', 'Gurgaon', 'Bangalore', 'Indore', 'Pune', 'Chennai', 'Mumbai', 'Hyderabad', 'Ahmedabad']

# bhk/rooms
city_room = ['Noida', 'Ghaziabad', 'Gurgaon', 'Delhi', 'Bangalore', 'Chennai', 'Mumbai', 'Indore', 'Nagpur']

In [11]:
information = []

Book Now Schedule Visit

Information present in each room: Image Link Rent for 1 --> class name --> Girls/ Boys/ Family BHK Area Rent/MOnth Security Deposit More Details --> Goes to new webpage

In [ ]:
def bhk(city):
    html_text = requests.get("https://www.roomsoom.com/flat/property-listing?lat=&lang=&city=" + city +
                             "&type=flat").text
    soup = BeautifulSoup(html_text, 'lxml')
    rooms = soup.find_all('div', class_ = "property_data")
    for room in rooms:
        image_link = room.img['src']
        people_allowed = room.find('div', class_ = "rent_for1").text
        bhk = room.find('div', class_ = "property_title").h3.text
        primitive_location = room.find('div', class_ = "property_title").p.text
        money_info = room.find_all('span', attrs={"class" : "pro_price"})
        rent_per_month = money_info[0].text
        security_deposit = money_info[1].text
        more_details_link = room.find('div', class_ = "col-md-2 col-xs-12").a['href']
        link = requests.get(more_details_link).text
        soup2 = BeautifulSoup(link, 'lxml')
        images1 = soup2.find('ul', class_ = "slides")
        images = []
        for image in images1.find_all('img'):
            images.append(image.get('src'))
        location = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[0]
        description1 = soup2.find('div', class_="col-md-6 col-sm-12 col-xs-12 border-right").ul
        description = []
        for desc in description1.find_all('li'):
            description.append(desc.text)
        description2 = soup2.find('div', class_="col-md-6 col-sm-12 col-xs-12").ul
        for desc in description2.find_all('li'):
            description.append(desc.text)
        amenities = soup2.find('div', attrs={"id" : "home_amenities"})
        amenities2 = amenities.find('div', class_ = "tab-content").text.replace('\n\n\n', '\n')
        amen = []
        amen = [y for y in (x.strip() for x in amenities2.splitlines()) if y]
        details = soup2.find('div', class_="rent_data").text.replace('\n', '')
        room_item = {
            "city": city,
            "living_type": "bhk/rooms",
            "image_front": image_link,
            "people": people_allowed,
            "bhk": bhk,
            "primitive_location": primitive_location,
            "rent_per_month": rent_per_month,
            "security_deposit": security_deposit,
            "more_details_link": more_details_link,
            "images_links": images,
            "description": description,
            "amenities": amen,
            "details": details,
        }
        information.append(room_item)

In [ ]:
def pg(city):
    url = "https://www.roomsoom.com/pg/property-listing?lat=&lang=&city=" + city + "&type=pg"
#     driver = webdriver.Chrome('/usr/bin/chromedriver') 
#     driver.get(url)
#     html_text = driver.page_source
#     html_text = requests.get(url).text
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("/usr/bin/chromedriver", chrome_options=options)
    driver.get(url)
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, 'lxml')
    rooms = soup.find_all('div', class_ = "property_data")
    for room in rooms:
        image_link = room.img['src']
        people_allowed = room.find('div', class_ = "rent_for1").text
        name = room.find('div', class_ = "property_title").h3.text
        primitive_location = room.find('div', class_ = "property_title").p.text
        money_info = room.find_all('span', attrs={"class" : "pro_price"})
        rent_per_month = money_info[0].text
        security_deposit = money_info[1].text
        more_details_link = room.find('div', class_ = "col-md-2 col-xs-12").a['href']
        link = requests.get(more_details_link).text
        soup2 = BeautifulSoup(link, 'lxml')
        images1 = soup2.find('ul', class_ = "slides")
        images = []
        for image in images1.find_all('img'):
            images.append(image.get('src'))
        location = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[0]
        number_of_beds = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[-1]
        description = []
        description1 = soup2.find('div', class_="col-md-4 border-right col-xs-6").ul
        description2 = soup2.find('div', class_="col-md-4 col-xs-6").ul
        result = "\n".join([" ".join(elem) for elem in zip(description1.text.split("\n"), description2.text.split("\n"))])
        result = result.replace('\n', '\n')
        result = ','.join(result.splitlines())
        result = result[2:-2]
        description.append(result)
        amenities = soup2.find('div', attrs={"id" : "home_amenities"})
        amenities2 = amenities.find('div', class_ = "tab-content").text.replace('\n\n\n', '\n')
        amen = []
        amen = [y for y in (x.strip() for x in amenities2.splitlines()) if y]
        details = soup2.find('div', class_="rent_data").text.replace('\n', '')
        room_item = {
            "city": city,
            "living_type": "pg/co-living",
            "image_front": image_link,
            "people": people_allowed,
            "name": name,
            "primitive_location": primitive_location,
            "rent_per_month": rent_per_month,
            "security_deposit": security_deposit,
            "total_beds": number_of_beds,
            "more_details_link": more_details_link,
            "images_links": images,
            "description": description,
            "amenities": amen,
            "details": details,
        }
        information.append(room_item)

In [ ]:
def pg(city):
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, 'lxml')
    rooms = soup.find_all('div', class_ = "property_data")
    for room in rooms:
        image_link = room.img['src']
        people_allowed = room.find('div', class_ = "rent_for1").text
        name = room.find('div', class_ = "property_title").h3.text
        primitive_location = room.find('div', class_ = "property_title").p.text
        money_info = room.find_all('span', attrs={"class" : "pro_price"})
        rent_per_month = money_info[0].text
        security_deposit = money_info[1].text
        more_details_link = room.find('div', class_ = "col-md-2 col-xs-12").a['href']
        link = requests.get(more_details_link).text
        soup2 = BeautifulSoup(link, 'lxml')
        images1 = soup2.find('ul', class_ = "slides")
        images = []
        for image in images1.find_all('img'):
            images.append(image.get('src'))
        location = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[0]
        number_of_beds = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[-1]
        description = []
        description1 = soup2.find('div', class_="col-md-4 border-right col-xs-6").ul
        description2 = soup2.find('div', class_="col-md-4 col-xs-6").ul
        result = "\n".join([" ".join(elem) for elem in zip(description1.text.split("\n"), description2.text.split("\n"))])
        result = result.replace('\n', '\n')
        result = ','.join(result.splitlines())
        result = result[2:-2]
        description.append(result)
        amenities = soup2.find('div', attrs={"id" : "home_amenities"})
        amenities2 = amenities.find('div', class_ = "tab-content").text.replace('\n\n\n', '\n')
        amen = []
        amen = [y for y in (x.strip() for x in amenities2.splitlines()) if y]
        details = soup2.find('div', class_="rent_data").text.replace('\n', '')
        room_item = {
            "city": city,
            "living_type": "pg/co-living",
            "image_front": image_link,
            "people": people_allowed,
            "name": name,
            "primitive_location": primitive_location,
            "rent_per_month": rent_per_month,
            "security_deposit": security_deposit,
            "total_beds": number_of_beds,
            "more_details_link": more_details_link,
            "images_links": images,
            "description": description,
            "amenities": amen,
            "details": details,
        }
        information.append(room_item)

In [ ]:
for city in city_room:
    bhk(city)

for city in city_pg:
    pg(city)

In [ ]:
len(information)

In [ ]:
df = pd.DataFrame(information)

In [ ]:
df.to_csv("Roomsoom_data.csv", index=None)

In [9]:
url = "https://www.roomsoom.com/pg/property-listing?lat=&lang=&city=Noida&type=pg"
driver = webdriver.Chrome("/usr/bin/chromedriver")
driver.get(url)
driver.maximize_window()
for i in range(0, 3):
    time.sleep(5)
    i = i + 1
    pg(Noida)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
#     driver.execute_script("window.scrollBy(0, 500)", "")
    time.sleep(5)
#     driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
     